In [26]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import os
import json


import pandas as pd
import ast
import re

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\minh1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
pd.read_csv("employee_information_raw.csv")

,Name,Nationality,DOB,Sex,Seniority,About,Education,Languages,Programing Language,Tool,Automation/Orchestration tool,Microservice,OS,Database,Public Cloud,Projects Experience,Certifications
0,Vu Viet A,Viet Nam,1993,Male,7+ years,4+ years of experience in Operation System. Kn...,"{'university': 'Academy of Fice', 'years': '20...","['Vietnamese', 'English']",NaN,"['Redmine', 'Grafana', 'Jira', 'Zabbix', 'MS O...","['Windows', 'Linux']",NaN,"['Windows', 'Linux']","['SQL Server', 'My SQL']",NaN,"[{""project name"": ""FCCL"", ""customer"": ""Japanes...",NaN
1,Bui Duc A,Viet Nam,1990,Male,NaN,2 years of experience in the Cloud Computing. ...,{'university': 'HoChiMinh City University of I...,English,"['Python', 'Bash Script']","['Grafana', 'Prometheus', 'NodeExporter', 'AWS...","['AWS Codepipeline', 'CodeBuild', 'GitlabCI', ...","['Docker', 'Kurbernetes']","['Windows', 'Linux', 'Ubuntu']",PostgresSQL,AWS,"[{""project name"": ""USHM"", ""customer"": ""Japan c...",NaN
2,To Quy T,Viet Nam,2003,Male,< 1 year,Skilled in developing Bidata platform on AWS. ...,{'university': 'HAUI Hanoi University of Indus...,"['Vietnamese', 'English']","['Python', 'C++', 'SQL', 'Bash Script', 'Scala...","['Databrick', 'Apache Spark', 'AWS Glue', 'AWS...",NaN,"['Kafka', 'Docker']","['Windows', 'Linux']","['MySQL', 'DynamoDB']",['AWS'],"[{""project name"": ""USMH"", ""customer"": ""Japanes...",NaN
3,Pham Viet H,Viet Nam,1993,Male,5 years,8 years of experience in the computer software...,{'university': 'Hanoi University of Science an...,"['Vietnamese', 'Japanese', 'English']","['Python', 'Bash Script']","['Grafana', 'Prometheus', 'Graylog']","['AWS CloudFormation', 'Ansible', 'Jenkins', '...","['Docker', 'ECS', 'ECR', 'Kurbernetes']","['Windows', 'Linux']","['MySQL', 'MongoDB', 'Oracle', 'DMS', 'Athena'...","['AWS', 'Azure', 'GCP']","[{""project name"": ""Cloud Group Leader"", ""custo...","['AWS Certified DevOps Engineer Professional',..."
4,Doan Dinh Vu C,Viet Nam,1997,Male,3.5 years,"I am a responsible, dedicated and proactive pe...","{'university': 'FPT University', 'years': '201...","['Vietnamese', 'Japanese', 'English']","['Java Spring Boost', 'Java Core', 'C#']","['Visual Studio Code', 'Eclipse', 'IntelliJ', ...",['Selenium'],NaN,['Windows'],"['MS SQL', 'My SQL', 'Postgre SQL']",['Nifcloud'],"[{""project name"": ""FCCL_Monitor_D3"", ""customer...",NaN
5,Le Huu M,Viet Nam,1994,Male,6+ years,6 years of experience in Operation System.Know...,"{'university': 'FPT Polytechnic', 'years': 'Now'}","['Vietnamese', 'English']",NaN,"['Redmine', 'Jira', 'CheckMK', 'Zabbix', 'Graf...",NaN,NaN,"['Windows', 'Linux']","['SQL Server', 'MS SQL']",['AWS'],"[{""project name"": ""FCCL"", ""customer"": ""Japanes...",NaN
6,Le Sy Q,Viet Nam,1993,Male,7+ years,"4+ years of experience in Developer, 2+ years ...",{'university': 'Hanoi University of Mining and...,"['Vietnamese', 'English']",NaN,['Redmine'],['Postman'],NaN,NaN,"['SQL Server', 'My SQL', 'PostgreSQL']",NaN,"[{""project name"": ""FCCL"", ""customer"": ""Japanes...",NaN
7,Ngo Thach A,Viet Nam,1994,Male,2 years,"2 year of experience in Operation System, Know...",{'university': 'Nam Dinh Industrial College Ni...,"['Vietnamese', 'Japanese', 'English']",NaN,"['Redmine', 'Grafana', 'IntelliJ', 'Visual Stu...",NaN,NaN,"['Windows', 'Linux']","['My SQL', 'Oracle']",NaN,"[{""project name"": ""FCCL"", ""customer"": ""Japanes...",NaN
8,Nguyen Phi Hai N,Viet Nam,2001,Male,3+ years,2+ years of experience in Website Testing Mobi...,"{'university': 'Hanoi University', 'years': '2...","{'Vietnamese': 'Native', 'English': 'C1 Advanc...",NaN,"Redmine, Backlog, MS Office","Postman, Katalon Studio, Test Cafe",NaN,"Windows, Centos 7","SQL Server, My SQL, PostgreSQL, DBeaver, MongoDB",AWS,"[{""project name"": ""EXV_Installation_Checksheet...",NaN
9,NGUYEN THI HONG N,Viet Nam,2000,Female,2 years,Over 2 years of experience working as a sub-PM...,{'University': 'University of Languages & Inte...,"['Vietnamese', 'Japanese', 'English']",['HTML/

In [28]:
lemmatizer = WordNetLemmatizer()

def standardize_seniority(entry):
    if not entry or pd.isna(entry):  # Kiểm tra nếu entry không có giá trị hoặc là NaN
        return None
    try:
        parsed = ast.literal_eval(entry)  # Thử chuyển entry thành kiểu dữ liệu Python hợp lệ
    except Exception:
        parsed = entry  # Nếu không thành công, giữ nguyên entry
    if isinstance(parsed, str):  # Nếu parsed là chuỗi
        parsed = re.sub(r'(?<=\d)\.(?=\d)', 'DOTPLACEHOLDER', parsed)  # Thay thế dấu chấm giữa các số bằng "DOTPLACEHOLDER"
        parsed = re.sub(r'[^\w\s]', '', parsed)  # Loại bỏ các ký tự đặc biệt không phải là chữ cái hoặc dấu cách
        parsed = parsed.replace('DOTPLACEHOLDER', '.')  # Khôi phục lại dấu chấm đã thay thế
        tokens = parsed.split()  # Tách chuỗi thành các từ
        new_tokens = []
        for token in tokens:
            if token.isalpha():  # Nếu token là chữ cái
                lemma = lemmatizer.lemmatize(token.lower(), pos='n')  # Lematize từ ở dạng số nhiều (danh từ)
                new_tokens.append(lemma)
            else:
                # Giữ nguyên các token là số hoặc có ký tự pha trộn
                new_tokens.append(token)
        return " ".join(new_tokens)  # Kết hợp lại các từ đã xử lý thành chuỗi
    else:
        return None  # Nếu parsed không phải là chuỗi, trả về None


In [29]:
def standardize(entry):
    # Xử lý trường hợp dữ liệu thiếu hoặc rỗng
    if not entry or pd.isna(entry):
        return {}

    #chuyển chuỗi "" thành đối tượng Python ví dụ như chuỗi "['list','list1']" sẽ thành dạng list : ['list','list1']
    try:
        parsed = ast.literal_eval(entry)
    except Exception:
        parsed = entry  # Nếu không thể chuyển thì coi đây là một chuỗi đơn giản

    # Nếu là dictionary, giữ nguyên các giá trị ban đầu (levels)
    if isinstance(parsed, dict):
        return parsed

    # Nếu là list, kiểm tra kiểu của các phần tử trong danh sách
    elif isinstance(parsed, list):
        # Nếu là list các chuỗi (ví dụ: ['Vietnamese', 'English'])
        if all(isinstance(item, str) for item in parsed):
            text = []
            for item in parsed:
                # Nếu chuỗi chứa dấu phẩy, tách nó thành nhiều phần
                if ',' in item:
                    text.extend([s.strip() for s in item.split(',') if s.strip()])
                elif '/' in item:
                    text.extend([s.strip() for s in item.split('/') if s.strip()])
                else:
                    text.append(item.strip())
            return {items: None for items in text}
        else:
            return {}

    # Nếu là một chuỗi đơn giản
    elif isinstance(parsed, str):
        if ',' in parsed:
            text = [s.strip() for s in parsed.split(',') if s.strip()]
            return {word: None for word in text}
        elif '/' in parsed:
            text = [s.strip() for s in parsed.split('/') if s.strip()]
            return {word: None for word in text}
        else:
            return {parsed.strip(): None}

    else:
        return {}


In [30]:
def standardize_education(entry):
    """
    Chuẩn hóa dữ liệu cột giáo dục.
    - Xử lý cả dictionary đơn lẻ và danh sách các dictionary
    - Giữ nguyên cấu trúc nhưng đảm bảo định dạng nhất quán
    """
    if not entry or pd.isna(entry):
        return {}

    try:
        parsed = ast.literal_eval(entry)  # Chuyển chuỗi thành đối tượng Python
    except Exception:
        # Nếu chuyển không thành công, trả về dictionary rỗng
        return {}

    # Xử lý trường hợp là một dictionary đơn lẻ
    if isinstance(parsed, dict):
        # Đảm bảo tất cả các khóa (key) đều ở dạng chữ thường
        return {k.lower(): v for k, v in parsed.items()}

    # Xử lý trường hợp là danh sách các dictionary
    elif isinstance(parsed, list) and all(isinstance(item, dict) for item in parsed):
        # Chuẩn hóa mỗi dictionary trong danh sách
        return [{k.lower(): v for k, v in item.items()} for item in parsed]

    # Nếu không phải là dictionary hoặc danh sách dictionary, trả về dictionary rỗng
    return {}


In [31]:
def standardize_certifications(entry):
    """
    Chuẩn hóa dữ liệu cột chứng chỉ.
    - Xử lý cả danh sách các chuỗi và danh sách các dictionary
    - Giữ nguyên cấu trúc nhưng đảm bảo định dạng nhất quán
    """
    if not entry or pd.isna(entry):
        return []  # Nếu dữ liệu rỗng hoặc NaN, trả về danh sách rỗng

    try:
        parsed = ast.literal_eval(entry)  # Chuyển chuỗi thành đối tượng Python
    except Exception:
        # Nếu chuyển không thành công, trả về danh sách rỗng
        return []

    # Xử lý trường hợp là danh sách các chuỗi
    if isinstance(parsed, list):
        if all(isinstance(item, str) for item in parsed):
            return parsed  # Nếu tất cả các phần tử là chuỗi, giữ nguyên danh sách

        # Xử lý trường hợp là danh sách các dictionary
        elif all(isinstance(item, dict) for item in parsed):
            # Chuẩn hóa mỗi dictionary trong danh sách bằng cách chuyển khóa thành chữ thường
            return [{k.lower(): v for k, v in item.items()} for item in parsed]

    # Xử lý trường hợp là một chuỗi đơn lẻ (hiếm nhưng có thể xảy ra)
    elif isinstance(parsed, str):
        return [parsed]  # Nếu là chuỗi, đóng gói nó trong một danh sách

    return []  # Nếu không phải dạng đã dự đoán, trả về danh sách rỗng


In [32]:
def fix_json_string(json_str):
    """
    Tiền xử lý chuỗi JSON:
    - Chuyển dấu nháy đơn thành dấu nháy kép
    - Thay thế None bằng null
    """
    # Chuyển đổi dấu nháy đơn thành dấu nháy kép
    fixed = json_str.replace("'", '"')
    # Thay thế None (không phân biệt hoa thường) bằng null
    fixed = fixed.replace("None", "null")
    return fixed

def process_projects_experience(pe):
    """
    Xử lý chuỗi trong cột Projects Experience:
    - Nếu rỗng hoặc '[]', trả về chính nó
    - Tiền xử lý chuỗi JSON, validate và trả về chuỗi JSON chuẩn (nếu được)
    """
    if pd.isna(pe) or pe.strip() == "" or pe.strip() == "[]":
        return pe
    try:
        fixed = fix_json_string(pe)
        # Thử parse chuỗi JSON
        data = json.loads(fixed)
        # Nếu parse thành công, chuyển về dạng JSON chuẩn (với dấu nháy kép, escape đúng cách)
        return json.dumps(data, ensure_ascii=False)
    except Exception as e:
        print("Error processing Projects Experience:", e)
        print("Dữ liệu gốc:", pe)
        # Nếu lỗi, trả về chuỗi đã được fix (có thể cần xử lý thêm)
        return fixed

In [33]:
df = pd.read_csv('employee_information_raw.csv')
output_file = "employee_information.csv"

df[['Languages', 'Programing Language','Tool','Automation/Orchestration tool','Microservice','OS','Database','Public Cloud']] = df[['Languages', 'Programing Language','Tool','Automation/Orchestration tool','Microservice','OS','Database','Public Cloud']].map(standardize)

df['Seniority'] = df['Seniority'].apply(standardize_seniority)
df["Projects Experience"] = df["Projects Experience"].apply(process_projects_experience)

df['Education'] = df['Education'].apply(standardize_education)
df['Certifications'] = df['Certifications'].apply(standardize_certifications)

df.to_csv('employee_information.csv', index=False)
print("Tiền xử lý CSV hoàn tất. File đã được lưu tại:", output_file)


Tiền xử lý CSV hoàn tất. File đã được lưu tại: employee_information.csv


In [34]:
pd.read_csv("employee_information.csv", encoding='UTF-8')

,Name,Nationality,DOB,Sex,Seniority,About,Education,Languages,Programing Language,Tool,Automation/Orchestration tool,Microservice,OS,Database,Public Cloud,Projects Experience,Certifications
0,Vu Viet A,Viet Nam,1993,Male,7 year,4+ years of experience in Operation System. Kn...,"{'university': 'Academy of Fice', 'years': '20...","{'Vietnamese': None, 'English': None}",{},"{'Redmine': None, 'Grafana': None, 'Jira': Non...","{'Windows': None, 'Linux': None}",{},"{'Windows': None, 'Linux': None}","{'SQL Server': None, 'My SQL': None}",{},"[{""project name"": ""FCCL"", ""customer"": ""Japanes...",[]
1,Bui Duc A,Viet Nam,1990,Male,NaN,2 years of experience in the Cloud Computing. ...,{'university': 'HoChiMinh City University of I...,{'English': None},"{'Python': None, 'Bash Script': None}","{'Grafana': None, 'Prometheus': None, 'NodeExp...","{'AWS Codepipeline': None, 'CodeBuild': None, ...","{'Docker': None, 'Kurbernetes': None}","{'Windows': None, 'Linux': None, 'Ubuntu': None}",{'PostgresSQL': None},{'AWS': None},"[{""project name"": ""USHM"", ""customer"": ""Japan c...",[]
2,To Quy T,Viet Nam,2003,Male,1 year,Skilled in developing Bidata platform on AWS. ...,{'university': 'HAUI Hanoi University of Indus...,"{'Vietnamese': None, 'English': None}","{'Python': None, 'C++': None, 'SQL': None, 'Ba...","{'Databrick': None, 'Apache Spark': None, 'AWS...",{},"{'Kafka': None, 'Docker': None}","{'Windows': None, 'Linux': None}","{'MySQL': None, 'DynamoDB': None}",{'AWS': None},"[{""project name"": ""USMH"", ""customer"": ""Japanes...",[]
3,Pham Viet H,Viet Nam,1993,Male,5 year,8 years of experience in the computer software...,{'university': 'Hanoi University of Science an...,"{'Vietnamese': None, 'Japanese': None, 'Englis...","{'Python': None, 'Bash Script': None}","{'Grafana': None, 'Prometheus': None, 'Graylog...","{'AWS CloudFormation': None, 'Ansible': None, ...","{'Docker': None, 'ECS': None, 'ECR': None, 'Ku...","{'Windows': None, 'Linux': None}","{'MySQL': None, 'MongoDB': None, 'Oracle': Non...","{'AWS': None, 'Azure': None, 'GCP': None}","[{""project name"": ""Cloud Group Leader"", ""custo...","['AWS Certified DevOps Engineer Professional',..."
4,Doan Dinh Vu C,Viet Nam,1997,Male,3.5 year,"I am a responsible, dedicated and proactive pe...","{'university': 'FPT University', 'years': '201...","{'Vietnamese': None, 'Japanese': None, 'Englis...","{'Java Spring Boost': None, 'Java Core': None,...","{'Visual Studio Code': None, 'Eclipse': None, ...",{'Selenium': None},{},{'Windows': None},"{'MS SQL': None, 'My SQL': None, 'Postgre SQL'...",{'Nifcloud': None},"[{""project name"": ""FCCL_Monitor_D3"", ""customer...",[]
5,Le Huu M,Viet Nam,1994,Male,6 year,6 years of experience in Operation System.Know...,"{'university': 'FPT Polytechnic', 'years': 'Now'}","{'Vietnamese': None, 'English': None}",{},"{'Redmine': None, 'Jira': None, 'CheckMK': Non...",{},{},"{'Windows': None, 'Linux': None}","{'SQL Server': None, 'MS SQL': None}",{'AWS': None},"[{""project name"": ""FCCL"", ""customer"": ""Japanes...",[]
6,Le Sy Q,Viet Nam,1993,Male,7 year,"4+ years of experience in Developer, 2+ years ...",{'university': 'Hanoi University of Mining and...,"{'Vietnamese': None, 'English': None}",{},{'Redmine': None},{'Postman': None},{},{},"{'SQL Server': None, 'My SQL': None, 'PostgreS...",{},"[{""project name"": ""FCCL"", ""customer"": ""Japanes...",[]
7,Ngo Thach A,Viet Nam,1994,Male,2 year,"2 year of experience in Operation System, Know...",{'university': 'Nam Dinh Industrial College Ni...,"{'Vietnamese': None, 'Japanese': None, 'Englis...",{},"{'Redmine': None, 'Grafana': None, 'IntelliJ':...",{},{},"{'Windows': None, 'Linux': None}","{'My SQL': None, 'Oracle': None}",{},"[{""project name"": ""FCCL"", ""customer"": ""Japanes...",[]
8,Nguyen Phi Hai N,Viet Nam,2001,Male,3 year,2+ years of experience in Website Testing Mobi...,"{'university': 'Hanoi University', 'years': '2...","{'Vietnamese': 'Native', 'English': 'C1 Advanc...",{},"{'Redmine': None, 'B